In [1]:
import os

In [2]:
os.chdir("../")
%pwd

'f:\\ProjectAI\\FaceEmotionRecognitionSystem'

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path 
    updated_base_model_path: Path
    training_data_path: Path
    dataset_path: Path
    model_name: str
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_learning_rate: float
    params_num_classes: int
    params_image_size: list

In [4]:
from FacialExpressionRecognition.constants import *
from FacialExpressionRecognition.utils.common import read_yaml, create_directories

In [5]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_training_config(self) -> TrainingConfig:
        training_config = self.config.training
        artifacts_root = self.config.artifacts_root
        trained_model_path = os.path.join(training_config.trained_model_path)
        updated_base_model_path = os.path.join(self.config.prepare_base_model.updated_base_model_path)
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "data.zip")
        dataset_path = os.path.join(training_config.dataset_path)

        create_directories([Path(training_config.root_dir)])

        params = self.params
        params_epochs = params.EPOCHS
        params_batch_size = params.BATCH_SIZE
        params_is_augmentation = params.AUGMENTATION
        params_learning_rate = params.LEARNING_RATE
        params_num_classes = params.NUM_CLASSES
        params_image_size = params.IMAGE_SIZE
        params_model_name = params.MODEL_NAME

        training_config = TrainingConfig(
            root_dir=Path(artifacts_root),
            trained_model_path=Path(trained_model_path),
            updated_base_model_path=Path(updated_base_model_path),
            training_data_path=Path(training_data),
            dataset_path=Path(dataset_path),
            model_name=params_model_name,
            params_epochs=params_epochs,
            params_batch_size=params_batch_size,
            params_is_augmentation=params_is_augmentation,
            params_learning_rate=params_learning_rate,
            params_num_classes=params_num_classes,
            params_image_size=params_image_size
        )

        return training_config

In [6]:
import os
import torch
import torchvision.models as models
import torchvision
import time
import tqdm
from zipfile import ZipFile
from FacialExpressionRecognition import logger
from FacialExpressionRecognition.models.resnet34 import get_resnet34_model


In [7]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config
    
    def get_model(self):
        if self.config.model_name == "resnet34":
            self.model = get_resnet34_model(num_classes=self.config.params_num_classes)
            self.model.load_state_dict(torch.load(self.config.updated_base_model_path))
            self.model.fc = torch.nn.Linear(self.model.fc.in_features, self.config.params_num_classes)
            self.model.eval()
        else:
            raise ValueError(f"Model {self.config.model_name} not supported.")
        

    def extract_dataset(self):
        with ZipFile(self.config.training_data_path, 'r') as zip_ref:
            os.makedirs(self.config.dataset_path, exist_ok=True)
            zip_ref.extractall(self.config.dataset_path)

    def train(self):
        train_transforms = torchvision.transforms.Compose([
            torchvision.transforms.Resize((self.config.params_image_size[0], self.config.params_image_size[1])),
            torchvision.transforms.ToTensor()
        ])

        self.extract_dataset()
        
        train_data = torchvision.datasets.ImageFolder(root=self.config.dataset_path, transform=train_transforms)
        train_loader = torch.utils.data.DataLoader(train_data, batch_size=self.config.params_batch_size, shuffle=True)

        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.get_model()
        self.model = self.model.to(device)

        criterion = torch.nn.CrossEntropyLoss()
        optimizer = torch.optim.Adam(self.model.parameters(), lr=self.config.params_learning_rate)

        num_epochs = self.config.params_epochs

        for epoch in range(num_epochs):
            self.model.train()
            running_loss = 0.0
            for images, labels in tqdm.tqdm(train_loader):
                images, labels = images.to(device), labels.to(device)

                optimizer.zero_grad()

                outputs = self.model(images)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()

                running_loss += loss.item() * images.size(0)

            epoch_loss = running_loss / len(train_loader.dataset)
            print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}')

        Training.save_model(self.model, self.config.trained_model_path)

    @staticmethod
    def save_model(model, path):
        torch.save(model, path)
        logger.info(f"Model saved at {path}")

In [9]:
try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_model()
    training.train()
    logger.info("Training completed successfully.")
except Exception as e:
    logger.exception(e)
    raise e

[2025-09-28 10:16:37,187: INFO: common]: yaml file: config\config.yaml loaded successfully
[2025-09-28 10:16:37,198: INFO: common]: yaml file: params.yaml loaded successfully
[2025-09-28 10:16:37,199: INFO: common]: Directory created at: artifacts
[2025-09-28 10:16:37,200: INFO: common]: Directory created at: artifacts\training


100%|██████████| 561/561 [09:56<00:00,  1.06s/it]

Epoch [1/1], Loss: 0.0035
[2025-09-28 10:26:43,975: INFO: 3183951286]: Model saved at artifacts\training\model.h5
[2025-09-28 10:26:43,987: INFO: 4016484038]: Training completed successfully.
